In [1]:
using JuMP
using Gurobi
using MixedIntegerExperiments
using Polyhedra
using CDDLib
using Plots
using DrakeVisualizer
using AxisArrays
# using BenchmarkTools

In [2]:
# Pkg.clone("https://github.com/andreasnoack/LinearAlgebra.jl")
# Pkg.clone("git@github.com:JuliaPolyhedra/ConvexHull.jl.git")

In [3]:
verbose = true;
show_plots = false;

In [4]:
# parameters
nsteps = 100
ncontacts = 2
h = 0.1
I = 1.
m = 10.
g = [0; -9.81]
r0 = [0.5; 0.8]
ṙ0 = [0.0; 0.0]
θ0 = 0.
ω0 = 0.
contactregion = contact_region([0.; 0.], [1.; 0.], 0.8, 2 * m * norm(g))
freeregion = axis_aligned_free_box_region([0.; 0.], [1.; 1.])
regions = [contactregion; freeregion]
nregions = length(regions);

In [5]:
if show_plots
    plt = plot()
    plot_polyhedron!(polyhedron(contactregion.position); lab = "contact", fillcolor = "black")
    plot_polyhedron!(polyhedron(freeregion.position); lab = "free", opacity = 0.5)
    title!(plt, "Environment")
    plt
end

In [6]:
if show_plots
    plt = plot()
    plot_polyhedron!(polyhedron(contactregion.force); opacity = 0.5, lab = "Contact region")
    title!("Allowable contact forces")
end

In [7]:
model = Model(solver = GurobiSolver(Presolve = 0, OutputFlag = Int(verbose)));

In [8]:
dimaxis = Axis{:dim}([:x, :z])
stepaxis = Axis{:step}(1 : nsteps)
contactaxis = Axis{:contact}(1 : nsteps)

AxisArrays.Axis{:contact,UnitRange{Int64}}(1:100)

In [15]:
function create_variables(model::Model, basename::String, axes::Axis...)
    ranges = map(axis -> 1 : length(axis), axes)
    vars = @variable(model, $ranges, basename = basename)
    AxisArray(vars, axes...)
end

create_variables (generic function with 1 method)

In [10]:
ranges = map(axis -> 1 : length(axis), (dimaxis, stepaxis))

(1:2,1:100)

In [11]:
vars = @variable(model, $ranges, basename = "r")

r[i] free ∀ i ∈ {1:2,1:100}

In [20]:
supertype(typeof(vars))

JuMP.JuMPContainer{JuMP.Variable,N}

In [16]:
bla = create_variables(model, "r", dimaxis, stepaxis);

LoadError: MethodError: no method matching AxisArrays.AxisArray{T,N,D,Ax}(::JuMP.JuMPArray{JuMP.Variable,1,Tuple{Tuple{UnitRange{Int64},UnitRange{Int64}}}}, ::AxisArrays.Axis{:dim,Array{Symbol,1}}, ::AxisArrays.Axis{:step,UnitRange{Int64}})[0m
Closest candidates are:
  AxisArrays.AxisArray{T,N,D,Ax}([1m[31m::AbstractArray{T,N}[0m, ::AxisArrays.Axis{name,T}...) at /home/twan/code/MixedIntegerExperiments/v0.5/AxisArrays/src/core.jl:176
  AxisArrays.AxisArray{T,N,D,Ax}{T}(::Any) at sysimg.jl:53[0m

In [50]:
@variable(model, [i = 1 : 3], basename = "r")

3-element Array{JuMP.Variable,1}:
 r[1]
 r[2]
 r[3]

In [41]:
rs = AxisArray(@variable(model, r[k = 1 : 2, n = 1 : nsteps]), dimaxis, stepaxis)
θs = AxisArray(@variable(model, θ[n = 1 : nsteps]), stepaxis)
ṙs = AxisArray(@variable(model, ṙ[k = 1 : 2, n = 1 : nsteps]), dimaxis, stepaxis)
ωs = AxisArray(@variable(model, ω[n = 1 : nsteps]), stepaxis)
rcs = AxisArray(@variable(model, rc[k = 1 : 2, i = 1 : ncontacts, n = 1 : nsteps]), dimaxis, contactaxis, stepaxis)
fs = AxisArray(@variable(model, f[k = 1 : 2, i = 1 : ncontacts, n = 1 : nsteps]), dimaxis, contactaxis, stepaxis)
F = AxisArray(@variable(model, F[k = 1 : 2, n = 1 : nsteps]), dimaxis, stepaxis)
Ts = AxisArray(@variable(model, T[n = 1 : nsteps]), stepaxis)
ss = AxisArray(@variable(model, s[k = 1 : 2, i = 1 : ncontacts, n = 1 : nsteps]), dimaxis, contactaxis, stepaxis)



2-dimensional AxisArray{JuMP.Variable,2,...} with axes:
    :dim, Symbol[:x,:z]
    :step, 1:100
And data, a 2×100 Array{JuMP.Variable,2}:
 r[1,1]  r[1,2]  r[1,3]  r[1,4]  …  r[1,97]  r[1,98]  r[1,99]  r[1,100]
 r[2,1]  r[2,2]  r[2,3]  r[2,4]     r[2,97]  r[2,98]  r[2,99]  r[2,100]

In [9]:
@variables(model, begin
    r[k = 1 : 2, n = 1 : nsteps]
    θ[n = 1 : nsteps]
    ṙ[k = 1 : 2, n = 1 : nsteps]
    ω[n = 1 : nsteps]
    rc[k = 1 : 2, i = 1 : ncontacts, n = 1 : nsteps]
    f[k = 1 : 2, i = 1 : ncontacts, n = 1 : nsteps]
    F[k = 1 : 2, n = 1 : nsteps]
    T[n = 1 : nsteps]
    s[k = 1 : 2, i = 1 : ncontacts, n = 1 : nsteps]
    wzx[i = 1 : ncontacts, n = 1 : nsteps]
    wxz[i = 1 : ncontacts, n = 1 : nsteps]
    γ[i = 1 : ncontacts, j = 1 : nregions, n = 1 : nsteps - 1]
    Γ[i = 1 : ncontacts, j = 1 : nregions]
end)

In [23]:
typeof(r)

Array{JuMP.Variable,2}

2-dimensional AxisArray{JuMP.Variable,2,...} with axes:
    :dim, Symbol[:x,:z]
    :step, 1:100
And data, a 2×100 Array{JuMP.Variable,2}:
 r[1,1]  r[1,2]  r[1,3]  r[1,4]  …  r[1,97]  r[1,98]  r[1,99]  r[1,100]
 r[2,1]  r[2,2]  r[2,3]  r[2,4]     r[2,97]  r[2,98]  r[2,99]  r[2,100]

In [38]:
typeof()

Array{JuMP.Variable,2}

In [10]:
# 4.6
z = Array{Variable, 3}(ncontacts, nregions, nsteps)
for i = 1 : ncontacts, n = 1 : nsteps
    position_force_regions = collect(cartesian_product(region.position, region.force) for region in regions)
    z[i, :, n] = hull_reformulation(model, position_force_regions, [rc[:, i, n]; f[:, i, n]])
end

In [11]:
@constraints(model, begin
    [n = 1 : nsteps, i = 1 : ncontacts, k = 1 : 2], s[k, i, n] == rc[k, i, n] - r[k, n]
end)

In [12]:
# 4.18
@constraints(model, begin
    [n = 1 : nsteps - 1, k = 1 : 2], r[k, n + 1] == r[k, n] + h * ṙ[k, n + 1]
    [n = 1 : nsteps - 1], θ[n + 1] == θ[n] + h * ω[n + 1]
    [n = 1 : nsteps - 1, k = 1 : 2], ṙ[k, n + 1] == ṙ[k, n] + h / m * F[k, n + 1]
    [n = 1 : nsteps - 1], ω[n + 1] == ω[n] + h / I * T[n + 1]
    [n = 1 : nsteps, k = 1 : 2], F[k, n] == m * g[k] + sum(f[k, i, n] for i = 1 : ncontacts)
    [n = 1 : nsteps], T[n] == sum(wzx[i, n] - wxz[i, n] for i = 1 : ncontacts)
end)

In [13]:
# convex bilinear term approximations (4.12, 4.13)
for i = 1 : ncontacts, n = 1 : nsteps
    # TODO: ranges, piecewise
    mccormick_envelope_constraints(model, wxz[i, n], s[1, i, n], f[2, i, n], -100., 100., -100., 100.)
    mccormick_envelope_constraints(model, wzx[i, n], s[2, i, n], f[1, i, n], -100., 100., -100., 100.)
end

In [14]:
# initial conditions
@constraints(model, begin
    r[:, 1] .== r0
    θ[1] == θ0
    ṙ[:, 1] .== ṙ0
    ω[1] == ω0
end)

In [15]:
# TODO: more task-specific constraints
@constraints(model, begin
    [n = 1 : nsteps], r[2, n] >= 0.
end)

In [16]:
# objective function + slack parameterization
@constraints(model, begin
    [i = 1 : ncontacts, j = 1 : nregions], Γ[i, j] == sum(γ[i, j, n] for n = 1 : nsteps - 1)
    [i = 1 : ncontacts, j = 1 : nregions, n = 1 : nsteps - 1], -γ[i, j, n] <= z[i, j, n + 1] - z[i, j, n]
    [i = 1 : ncontacts, j = 1 : nregions, n = 1 : nsteps - 1], z[i, j, n + 1] - z[i, j, n] <= γ[i, j, n]
    [i = 1 : ncontacts, j = 1 : nregions, n = 1 : nsteps - 1], 0 <= γ[i, j, n]
    [i = 1 : ncontacts, j = 1 : nregions, n = 1 : nsteps - 1], γ[i, j, n] <= 1
end)

# @objective(model, begin
#     sum(Γ[i, j] for i = 1 : ncontacts, j = 1 : nregions) + sum(f[]) # TODO
# end)

In [17]:
solve(model)

Optimize a model with 8188 rows, 4900 columns and 19156 nonzeros
Variable types: 4500 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+02]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 1e+04]
Variable types: 3896 continuous, 1004 integer (400 binary)

Root relaxation: objective 0.000000e+00, 3537 iterations, 0.29 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.0000000    0.00000      -     -    0s

Explored 0 nodes (3537 simplex iterations) in 0.35 seconds
Thread count was 20 (of 20 available processors)

Solution count 1: 0 
Pool objective bound 0

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap -


In [18]:
vis = MixedIntegerExperiments.plot_piecewise_mccormick(10, -1., 1., -1., 1.);

In [19]:
# delete!(vis)